In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import requests

In [3]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [4]:
data = requests.get(standings_url)

In [5]:
from bs4 import BeautifulSoup

In [6]:
soup = BeautifulSoup(data.text)

In [7]:
standings_table = soup.select('table.stats_table')[0]

In [8]:
links = standings_table.find_all('a')

In [9]:
links = [l.get("href") for l in links]

In [10]:
links = [l for l in links if '/squads/' in l]

In [11]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [12]:
team_url = team_urls[0]

In [13]:
data = requests.get(team_url)

In [14]:
import pandas as pd

In [15]:
matches = pd.read_html(data.text, match="Scores & Fixtures")

In [16]:
soup = BeautifulSoup(data.text)

In [17]:
links = soup.find_all('a')

In [18]:
links = [l.get("href") for l in links]

In [19]:
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [20]:
data = requests.get(f"https://fbref.com{links[0]}")

In [21]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [22]:
shooting.columns = shooting.columns.droplevel()

In [23]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt", "xG", "npxG"]], on="Date")

In [24]:
years = list(range(2024,2017, -1))

In [25]:
all_matches = []

In [26]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [27]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        time.sleep(4)
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt", "xG", "npxG"]], on="Date")
        except (ValueError, KeyError) as error:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(4)

In [28]:
match_df = pd.concat(all_matches)

In [29]:
match_df.columns = [c.lower() for c in match_df.columns]

In [31]:
match_df.to_csv("matches.csv")